In [1]:
import numpy as np
import requests
import json
from llm_models import run_chat
import openai
import time
from utils import *
from embeddings import *

c:\Users\LP00035\anaconda3\envs\openai\lib\site-packages\langchain\__init__.py:24: UserWarning: Importing OpenAI from langchain root module is no longer supported.
  warnings.warn(


In [2]:
#load telegram token
with open('../token_telegram.txt', 'r') as file:
    TOKEN = file.read().strip()

#load the openai key
with open('../openaikey.txt', 'r') as file:
    OPENAI_KEY = file.read().strip()

openai.api_key = OPENAI_KEY

In [3]:
# Función para iniciar un chat
def start(update, context):
    chat_id = update.message.chat_id
    if chat_id not in chat_models:
        chat_models[chat_id] = initialize_chat_model()  # Implementa esta función según tus necesidades
    context.bot.send_message(chat_id=chat_id, text="Chat running. Type 'exit' to quit.")

In [4]:
# define a function to get the message
def get_updates(offset):
    url = f"https://api.telegram.org/bot{TOKEN}/getUpdates"
    params = {"timeout": 100, "offset": offset}
    response = requests.get(url, params=params)
    return response.json()["result"]

In [5]:
# define a function to send a message
def send_message(chat_id, text):
    url = f"https://api.telegram.org/bot{TOKEN}/sendMessage"
    params = {"chat_id": chat_id, "text": text}
    response = requests.post(url, params=params)
    # return response

In [17]:
def main_2():
    print("Starting bot...")
    offset = 0
    # load vectorial data base
    db = get_vector_db('../data/historia.txt', 'e-sas_company')
    # define the retriever
    retriever = db.as_retriever(search_kwargs={'k': 5})
    # define the model
    llm = load_llm_model('../openaikey.txt', 300)
    # define a dictionary to save the memory of the models
    memory_users = {}
    while True:
        updates = get_updates(offset)
        if updates:
            for update in updates:
                offset = update["update_id"] + 1
                if "message" in update and "text" in update["message"]:
                    chat_id = update["message"]["chat"]["id"]
                    user_message = update["message"]["text"]
                    if chat_id not in memory_users:
                        memory_users[chat_id] = memory_object('../openaikey.txt')
                        send_message(chat_id, "Welcome to the chatbot of E-SAS. You can ask me about the command of the company.")
                    if user_message is not None and user_message == '/start':
                        pass
                    else:
                        response = process_query_with_memory(llm, retriever, user_message, memory_users[chat_id])
                        response = response['answer']
                        print(f"Response: {response}")
                        # send the message to the user
                        send_message(chat_id, response)
                        time.sleep(180)
        else:
            time.sleep(1)

In [18]:
main_2()

Starting bot...


Vector database is being loaded.

No sentence-transformers model found with name C:\Users\LP00035/.cache\torch\sentence_transformers\allenai_longformer-base-4096. Creating a new one with MEAN pooling.


Sebastián Sarasti es el fundador de E-SAS, una empresa relacionada con la tecnología y los productos para mascotas.
Es un emprendedor apasionado por la tecnología y los animales, especialmente los perros. Su historia con su 
compañero canino, Nikola, ha sido inspiradora para el crecimiento y éxito de su empresa.

Response: Sebastián Sarasti es el fundador de E-SAS, una empresa relacionada con la tecnología y los productos para mascotas. Es un emprendedor apasionado por la tecnología y los animales, especialmente los perros. Su historia con su compañero canino, Nikola, ha sido inspiradora para el crecimiento y éxito de su empresa.


...

The motivation behind E-SAS is a combination of the founder's passion for technology, programming, and artificial 
intelligence, as well as his unwavering love for animals. Sebastián Alejandro Sarasti, the founder, wanted to 
ensure that pet lovers in South America had access to high-quality pet products, regardless of where they lived. He
believed in the importance of accessibility and convenience, which led him to offer free shipping within Ecuador 
and establish a regional shipping system to make E-SAS a viable option for pet owners in other countries in South 
America.

Response: The motivation behind E-SAS is a combination of the founder's passion for technology, programming, and artificial intelligence, as well as his unwavering love for animals. Sebastián Alejandro Sarasti, the founder, wanted to ensure that pet lovers in South America had access to high-quality pet products, regardless of where they lived. He believed in the importance of accessibility and convenience, which led him to offer free shipping within Ecuador and establish a regional shipping system to make E-SAS a viable option for pet owners in other countries in South America.


No se menciona en el contexto que Sebastián esté intentando emular a Amazon.

Response: No se menciona en el contexto que Sebastián esté intentando emular a Amazon.


KeyboardInterrupt: 